In [1]:
setwd('~/single_cell_project/spatial/Trendsceek/')

In [2]:
library('trendsceek')
library("dplyr")
library('Seurat')
library('data.table')

Warning message:
“package ‘dplyr’ was built under R version 3.4.4”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2
Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”Loading required package: cowplot

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave

Loading required package: Matrix
Warning message:
“package ‘data.table’ was built under R version 3.4.4”
Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [3]:
winsorize <- function(x, n.win = 1){
###Winsorize extreme values
###Ex: Two most extreme values (from each side):
###t(apply(ed, 1, winsorize, 2))

    n.vals = length(x)
    fraction = n.win / n.vals
    if(length(fraction) != 1 || fraction < 0 || fraction > 0.5){
        stop("bad value for 'fraction'")
    }

    win.sorted.ind = order(x)
    x[win.sorted.ind[1:n.win]] = x[win.sorted.ind[n.win + 1]]
    x[win.sorted.ind[(n.vals - n.win + 1):n.vals]] = x[win.sorted.ind[n.vals - n.win]]

    ##fraction-based
    if(0){
        lim = stats::quantile(x, probs = c(fraction, 1 - fraction))

        ##set extreme values
        x[ x < lim[1] ] = lim[1]
        x[ x > lim[2] ] = lim[2]
    }
    
    return(x)
}

In [ ]:
for (i in 33:53){
#i=43
df=paste0('Data/field_',i,'_counts_0.csv')
data=read.csv(df,header = T,row.names=1)
counts=t(data)
win.counts=t(apply(counts,1,winsorize,4))
Rep<-genefilter_exprmat(counts,min.expr=5,min.ncells.expr = 3)
Rep <-CreateSeuratObject(raw.data = Rep ,min.cells = 3,min.genes = 5 )
Rep_filt<- FilterCells(Rep, subset.names = "nGene", low.thresholds = 500, high.thresholds = Inf)
n.gene=nrow(Rep_filt@raw.data)
n.cell=ncol(Rep_filt@raw.data)
Rep<-NormalizeData(Rep_filt,normalization.method = "LogNormalize",scale.factor = 10000)

info<-read.csv(paste0('Data/field_',i,'_sample_info_0.csv'),row.names=1)
corr<-data.frame(x=info$x,y=info$y)
dim(corr)
pp=pos2pp(corr)
pp=set_marks(pp,win.counts,log.fcn = log10,pseudo.count=1)
sub_pp=head(colnames(pp$marks),10)
#pp2plot
pp2plot=pp_select(pp,sub_pp)




#t0<-Sys.time() 
trendstat_list = trendsceek_test(pp)
#t1<-Sys.time()

sig_list_0.05 = extract_sig_genes(trendstat_list, alpha=0.05)
lapply(sig_list_0.05, nrow)

sig_genes<-sig_list_0.05$Emark$gene
p.bh<-sig_list_0.05$Emark$p.bh
df_E<-data.frame(sig_genes,p.bh)
fwrite(df_E,paste0('running_results/middle_resule/field_',i,'Emark_0.05.csv'))

sig_genes<-sig_list_0.05$markcorr$gene
p.bh<-sig_list_0.05$markcorr$p.bh
df_c<-data.frame(sig_genes,p.bh)
fwrite(df_c,paste0('running_results/middle_resule/field_',i,'markcorr_0.05.csv'))


    }